<a href="https://colab.research.google.com/github/kr7/dcnn-forecast/blob/main/DCNN_forecast_exchange_rate.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Note: this notebook can be directly executed in Google Colab

import numpy as np 
import random
import scipy
import torch
import torch.nn as nn
import torch.optim as optim

from numpy import genfromtxt

In [ ]:
!pip install gdown

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!gdown "https://drive.google.com/uc?id=1MGIl1Aqnek0rPoPyqgS_Wzo5eQgIjihh"

Downloading...
From: https://drive.google.com/uc?id=1MGIl1Aqnek0rPoPyqgS_Wzo5eQgIjihh
To: /content/exchange_rate.txt
100% 546k/546k [00:00<00:00, 121MB/s]


In [ ]:
INPUT_LENGTH = 32
FORECAST_HORIZON = 16

In [ ]:
file_name = "exchange_rate.txt"
data = np.genfromtxt(file_name, delimiter=',')
data = data.transpose()

In [ ]:
# split the long time series: using INPUT_LENGTH observations, we will forecast the next FORECAST_HORIZON observations
input_data = []
target = []

for i in range(len(data)):
  for j in range(0,len(data[i])-(INPUT_LENGTH+FORECAST_HORIZON), INPUT_LENGTH+FORECAST_HORIZON):
    input_data.append(list(data[i, j:j+INPUT_LENGTH]))
    target.append( list(data[i, j+INPUT_LENGTH:j+INPUT_LENGTH+FORECAST_HORIZON] ))

input_data = np.array(input_data)
target = np.array(target)

NUM_INPUT_FEATURES = INPUT_LENGTH

In [ ]:
%load_ext cython

In [ ]:
%%cython

import numpy as np
cimport numpy as np

def dtw(np.ndarray[np.float_t, ndim=1] ts1, 
        np.ndarray[np.float_t, ndim=1] ts2):
  
  cdef int LEN_TS1 
  cdef int LEN_TS2
  cdef int i
  cdef int j
  cdef np.ndarray[np.float_t, ndim=2] dtw_matrix

  LEN_TS1 = len(ts1)
  LEN_TS2 = len(ts2)

  dtw_matrix = np.zeros( (LEN_TS1, LEN_TS2), dtype=float )
  
  dtw_matrix[0,0] = abs(ts1[0]-ts2[0])
  
  for i in range(1, LEN_TS1):
    dtw_matrix[i,0] = dtw_matrix[i-1,0]+abs(ts1[i]-ts2[0])

  for j in range(1, LEN_TS2):
    dtw_matrix[0,j] = dtw_matrix[0,j-1]+abs(ts1[0]-ts2[j])

  for i in range(1, LEN_TS1):
    for j in range(1, LEN_TS2):
      dtw_matrix[i,j] = min(dtw_matrix[i-1,j-1], dtw_matrix[i-1,j], 
                            dtw_matrix[i, j-1]) + abs(ts1[i]-ts2[j])
  
  return dtw_matrix[ len(ts1)-1, len(ts2)-1 ]

In [ ]:
def dc_activations(data, convolutional_filters):
    """
    Calculation of the activations of the distortion-aware convolutional layer.

    Inputs
    ------
    data : np.array 
      Two-dimensional array containing the input data, 
      each row of the array corresponds to one of the time series
    convolutional_filters : np.array
      Three-dimensional array containing the parameters of the dynamic 
      convolutional layer. The first index corresponds to the output channel
      of the convolution, the second index corresponds to the input channel 
      (the current implementation only works with 1 input channel, so the second
      index is always zero), the third index is the position within the local
      pattern corresponding to a convolutional filter
    """
    num_instances = len(data)
    length_of_time_series = len(data[0])
    num_conv_filters = len(convolutional_filters)
    conv_filter_size = len(convolutional_filters[0][0])

    activations = np.zeros( (num_instances, 2*num_conv_filters, 
                           length_of_time_series-conv_filter_size+1) )
    for i in range(num_instances):
        for j in range(length_of_time_series-conv_filter_size+1):
            for k in range(num_conv_filters):
                activations[i,k,j] = 1.0/(1.0 + dtw(convolutional_filters[k][0],
                                            data[i,j:j+conv_filter_size]))
    for i in range(num_instances):
        for j in range(length_of_time_series-conv_filter_size+1):
            for k in range(num_conv_filters):
                activations[i,num_conv_filters+k,j] = np.sum(convolutional_filters[k][0]*data[i,j:j+conv_filter_size])

    return activations

In [ ]:
# Definition of the neural networks

CONV_FILTERS = 25
CONV_FILTER_SIZE = 9


class Net_CNN(nn.Module):
    def __init__(self):
        super(Net_CNN, self).__init__()
        num_units_fc = 100
        self.num_inputs_fc = int(CONV_FILTERS*(NUM_INPUT_FEATURES-CONV_FILTER_SIZE+1)/2)

        self.conv1 = nn.Conv1d(in_channels = 1, out_channels = CONV_FILTERS, 
                               kernel_size=CONV_FILTER_SIZE, padding = 0, stride = 1)
        self.max_pool = nn.MaxPool1d(2)
        self.fc = nn.Linear(self.num_inputs_fc, num_units_fc)
        self.out = nn.Linear(num_units_fc, FORECAST_HORIZON) 

    def forward(self, x):
        x = x.view(-1, 1, NUM_INPUT_FEATURES)
        x = self.conv1(x)
        x = self.max_pool(x)
        x = x.view(-1, self.num_inputs_fc)
        x = torch.relu(self.fc(x))
        x = self.out(x)
        return x


# Please note that the distortion-aware convolutional layer is initialized  
# using the parameters learned during the "initial stage" (in which a "usual" 
# convolutional network is trained). Once the initial stage is completed, 
# the parameters of the dynamic convoltuional layer are fixed, therefore,
# the activations of the dynamic convolutional layer will be pre-calculated 
# outside the network for efficient implementation.

class Net_DCNN(nn.Module):
    def __init__(self):
        super(Net_DCNN, self).__init__()
        num_units_fc = 100

        self.max_pool = nn.MaxPool1d(2)
        self.fc = nn.Linear(int(2*CONV_FILTERS*(NUM_INPUT_FEATURES-CONV_FILTER_SIZE+1)/2), num_units_fc)
        self.out = nn.Linear(num_units_fc, FORECAST_HORIZON) 

    def forward(self, x):
        x = x.view(-1, 2*CONV_FILTERS, NUM_INPUT_FEATURES-CONV_FILTER_SIZE+1)
        x = self.max_pool(x)
        x = x.view(-1,int(2*CONV_FILTERS*(NUM_INPUT_FEATURES-CONV_FILTER_SIZE+1)/2))
        x = torch.relu(self.fc(x))
        x = self.out(x)
        return x

In [ ]:
# function used to evaluate the network 
def eval_net(net, test_input, test_target):
    test_dataset = torch.utils.data.TensorDataset( 
        torch.Tensor(test_input), 
        torch.Tensor(test_target)
    )
    testloader = torch.utils.data.DataLoader(test_dataset, batch_size=1)

    mae = 0
    mse = 0
    total = 0

    with torch.no_grad():
        for inputs, true_target in testloader:
            inputs = inputs.cuda()
            true_target = true_target.cuda()
            predicted_target = net(inputs).cpu().numpy()
            true_target = true_target.cpu().numpy()

            mae += float(np.mean(np.abs(predicted_target - true_target)))
            mse += float(np.mean((predicted_target - true_target)**2))
            total += 1

    return mse/total, mae/total

In [ ]:
# functions used to distort time series

def atomic_elongational_noise(ts):
    n = len(ts)
    pos_del = random.randint(0,n-1)
    pos_elongate = random.randint(0,n-2)
    ts = list(ts)
    ts = ts[:pos_del] + ts[pos_del+1:] # delete a value
    ts = ts[:pos_elongate+1] + ts[pos_elongate:] # elongate a value
    return ts

def elongational_noise(timeseries_dataset, noise_level):
    timeseries_dataset_with_noise = []
    for i in range(len(timeseries_dataset)):
        ts = list(timeseries_dataset[i])
        for j in range(noise_level):
            ts = atomic_elongational_noise(ts)
        timeseries_dataset_with_noise.append(ts)
    return np.array(timeseries_dataset_with_noise)

In [ ]:
# the main experimental loop

from sklearn.model_selection import KFold

mse_cnn  = [[],[],[],[]]
mse_dcnn = [[],[],[],[]]
mae_cnn  = [[],[],[],[]]
mae_dcnn = [[],[],[],[]]
kf = KFold(n_splits=10, random_state=42, shuffle=True)

fold = 0
for train_index, test_index in kf.split(input_data):
    fold = fold + 1

    train_data = input_data[train_index]
    train_target = target[train_index]
    test_data = input_data[test_index]
    test_target = target[test_index]

    # Training of CNN. This is simultaneously the initial stage of training of the DCNN.

    train_dataset = torch.utils.data.TensorDataset(
      torch.Tensor(train_data), 
      torch.Tensor(train_target) 
    )
    trainloader = torch.utils.data.DataLoader(
      train_dataset, shuffle=True, batch_size=16)

    cnn = Net_CNN()
    cnn.cuda()
    criterion = nn.MSELoss()
    optimizer = optim.Adam(cnn.parameters(), lr=1e-5)

    running_loss = 0.0
    running_n = 0

    print("Training CNN...")

    for epoch in range(500):  
        for input_batch, target_batch in trainloader:
            input_batch = input_batch.cuda()
            target_batch = target_batch.cuda()

            optimizer.zero_grad()

            prediction_batch = cnn(input_batch)

            loss = criterion(prediction_batch, target_batch)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            running_n = running_n + 1

        if epoch % 100 == 0:
            print("epoch: {:3d} loss: {:4.3f}".format(epoch, running_loss/running_n))
            running_loss = 0.0
            running_n = 0

    # Obtain the parameters of the dynamic convolutional layer, and 
    # precalculate its activations
    params = []
    for p in cnn.parameters():
        params.append(p)

    convolutional_filters = np.array(params[0].cpu().detach().numpy(), dtype=float)
    dc_activations_train = dc_activations(train_data, convolutional_filters)

    # Train DCNN

    train_dataset = torch.utils.data.TensorDataset(
      torch.Tensor(dc_activations_train), 
      torch.Tensor(train_target) 
    )
    trainloader = torch.utils.data.DataLoader(
      train_dataset, shuffle=True, batch_size=16)


    dcnn = Net_DCNN()
    dcnn.cuda()
    criterion = nn.MSELoss()
    optimizer = optim.Adam(dcnn.parameters(), lr=1e-5)

    running_loss = 0.0
    running_n = 0

    print("Training DCNN...")

    for epoch in range(500):  
        for input_batch, target_batch in trainloader:
            input_batch = input_batch.cuda()
            target_batch = target_batch.cuda()

            optimizer.zero_grad()

            prediction_batch = dcnn(input_batch) 

            loss = criterion(prediction_batch, target_batch)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            running_n = running_n + 1

        if epoch % 100 == 0:
            print("epoch: {:3d} loss: {:4.3f}".format(epoch, running_loss/running_n))
            running_loss = 0.0
            running_n = 0
  

    # Evaluation at different levels of distortion
    
    test_data1 = test_data_with_noise = elongational_noise(test_data, 1)
    test_data5 = test_data_with_noise = elongational_noise(test_data, 5)
    test_data10 = test_data_with_noise = elongational_noise(test_data, 10)
    
    testsets = [test_data, test_data1, test_data5, test_data10]
    noise_level = [0, 1, 5, 10]
    
    for i in range(4):
        a_mse, a_mae = eval_net(cnn, testsets[i], test_target)
        mse_cnn[i].append(a_mse)
        mae_cnn[i].append(a_mae)

        dc_activations_test = dc_activations(testsets[i], convolutional_filters)
        a_mse_dc, a_mae_dc = eval_net(dcnn, dc_activations_test, test_target)

        mse_dcnn[i].append(a_mse_dc)
        mae_dcnn[i].append(a_mae_dc)

        print(f"Fold: {fold:2d}, Distortion: {noise_level[i]}")
        print(f"  aMSE of CNN:  {a_mse:8.6f}")
        print(f"  aMSE of DCNN: {a_mse_dc:8.6f}")
        print(f"  aMAE of CNN:  {a_mae:8.6f}")
        print(f"  aMAE of DCNN: {a_mae_dc:8.6f}")

Training CNN...
epoch:   0 loss: 0.626
epoch: 100 loss: 0.024
epoch: 200 loss: 0.000
epoch: 300 loss: 0.000
epoch: 400 loss: 0.000
Training DCNN...
epoch:   0 loss: 0.665
epoch: 100 loss: 0.026
epoch: 200 loss: 0.000
epoch: 300 loss: 0.000
epoch: 400 loss: 0.000
Fold:  1, Distortion: 0
  aMSE of CNN:  0.000517
  aMSE of DCNN: 0.000480
  aMAE of CNN:  0.013764
  aMAE of DCNN: 0.013382
Fold:  1, Distortion: 1
  aMSE of CNN:  0.000514
  aMSE of DCNN: 0.000477
  aMAE of CNN:  0.013713
  aMAE of DCNN: 0.013335
Fold:  1, Distortion: 5
  aMSE of CNN:  0.000524
  aMSE of DCNN: 0.000490
  aMAE of CNN:  0.013766
  aMAE of DCNN: 0.013421
Fold:  1, Distortion: 10
  aMSE of CNN:  0.000534
  aMSE of DCNN: 0.000499
  aMAE of CNN:  0.013897
  aMAE of DCNN: 0.013566
Training CNN...
epoch:   0 loss: 0.654
epoch: 100 loss: 0.034
epoch: 200 loss: 0.000
epoch: 300 loss: 0.000
epoch: 400 loss: 0.000
Training DCNN...
epoch:   0 loss: 0.683
epoch: 100 loss: 0.033
epoch: 200 loss: 0.000
epoch: 300 loss: 0.000


In [ ]:
# print results, calculate p-values 

print(file_name)
print(f"Mean avgMSE CNN:   {np.mean(mse_cnn[0]):6.5f} {np.mean(mse_cnn[1]):6.5f} {np.mean(mse_cnn[2]):6.5f} {np.mean(mse_cnn[3]):6.5f}")
print(f"Mean avgMSE DCNN:  {np.mean(mse_dcnn[0]):6.5f} {np.mean(mse_dcnn[1]):6.5f} {np.mean(mse_dcnn[2]):6.5f} {np.mean(mse_dcnn[3]):6.5f}")
print(f"Std. avgMSE CNN:   {np.std(mse_cnn[0]):6.5f} {np.std(mse_cnn[1]):6.5f} {np.std(mse_cnn[2]):6.5f} {np.std(mse_cnn[3]):6.5f}")
print(f"Std. avgMSE DCNN:  {np.std(mse_dcnn[0]):6.5f} {np.std(mse_dcnn[1]):6.5f} {np.std(mse_dcnn[2]):6.5f} {np.std(mse_dcnn[3]):6.5f}")
print(f"p-value:           {scipy.stats.ttest_rel(mse_cnn[0], mse_dcnn[0])[1]:6.5f} "+
                        f" {scipy.stats.ttest_rel(mse_cnn[1], mse_dcnn[1])[1]:6.5f} "+
                        f" {scipy.stats.ttest_rel(mse_cnn[2], mse_dcnn[2])[1]:6.5f} "+
                        f" {scipy.stats.ttest_rel(mse_cnn[3], mse_dcnn[3])[1]:6.5f}")

print(f"Mean avgMAE CNN:   {np.mean(mae_cnn[0]):6.5f} {np.mean(mae_cnn[1]):6.5f} {np.mean(mae_cnn[2]):6.5f} {np.mean(mae_cnn[3]):6.5f}")
print(f"Mean avgMAE DCNN:  {np.mean(mae_dcnn[0]):6.5f} {np.mean(mae_dcnn[1]):6.5f} {np.mean(mae_dcnn[2]):6.5f} {np.mean(mae_dcnn[3]):6.5f}")
print(f"Std. avgMAE CNN:   {np.std(mae_cnn[0]):6.5f} {np.std(mae_cnn[1]):6.5f} {np.std(mae_cnn[2]):6.5f} {np.std(mae_cnn[3]):6.5f}")
print(f"Std. avgMAE DCNN:  {np.std(mae_dcnn[0]):6.5f} {np.std(mae_dcnn[1]):6.5f} {np.std(mae_dcnn[2]):6.5f} {np.std(mae_dcnn[3]):6.5f}")
print(f"p-value:           {scipy.stats.ttest_rel(mae_cnn[0], mae_dcnn[0])[1]:6.5f}"+
                        f" {scipy.stats.ttest_rel(mae_cnn[1], mae_dcnn[1])[1]:6.5f}"+
                        f" {scipy.stats.ttest_rel(mae_cnn[2], mae_dcnn[2])[1]:6.5f}"+
                        f" {scipy.stats.ttest_rel(mae_cnn[3], mae_dcnn[3])[1]:6.5f}")

exchange_rate.txt
Mean avgMSE CNN:   0.00045 0.00045 0.00045 0.00046
Mean avgMSE DCNN:  0.00043 0.00043 0.00043 0.00044
Std. avgMSE CNN:   0.00012 0.00013 0.00013 0.00012
Std. avgMSE DCNN:  0.00013 0.00013 0.00013 0.00012
p-value:           0.00258  0.00216  0.00512  0.00753
Mean avgMAE CNN:   0.01234 0.01236 0.01236 0.01246
Mean avgMAE DCNN:  0.01223 0.01225 0.01228 0.01239
Std. avgMAE CNN:   0.00132 0.00134 0.00133 0.00126
Std. avgMAE DCNN:  0.00164 0.00164 0.00165 0.00157
p-value:           0.60156 0.61029 0.70559 0.70663
